# Deploy a pretrained sentiment model from Watson NLP as a Python function

Disclaimer: This notebook is intended to be used with an environment based on software configuration _Runtime 23.1 on Python 3.10_.

This notebook shows the steps needed to create and deploy a Python function that predicts sentiment in Watson Machine Learning. the example creates an _online deployment_ of the Python function. This way, the sentiment analysis can be invoked via a web service in applications.

## Prerequisites 

Before running this notebook: Create a new deployment space, or identify an existing deployment space to use for this example

## Initialize the Watson Machine Learning client

In [1]:
CPD_URL = '<CPD PLATFORM URL>'

In [2]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [ ]:
wml_credentials = {
    "url": CPD_URL,
    "token": wslib.auth.get_current_token(),
    "instance_id": "wml_local",
    "version" : "4.7"
}

In [4]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [ ]:
client.spaces.list()

### Add the `ID` of the space to use for deploying the function

In [6]:
client.set.default_space('<SPACE ID>')

'SUCCESS'

## Create a Python function that detects sentiment

This function is using the targeted sentiment block `targets-sentiment_transformer-workflow_multilingual_slate.153m.distilled-cpu` provided by Watson NLP.

In [7]:
def detect_sentiment():
    import watson_nlp
    
    sentiment_model = targets_sentiment_model = watson_nlp.load('targets-sentiment_transformer-workflow_multilingual_slate.153m.distilled-cpu')
    
    def score(input):
        scoring_prediction_out = []
        for input_data_row in input["input_data"][0]["values"]: 
            scoring_prediction_row = []
            for input_data in input_data_row:            
                targets_sentiments = targets_sentiment_model.run(input_data)
                scoring_prediction_row.append(targets_sentiments.to_dict())
                
            scoring_prediction_out.append(scoring_prediction_row)
            
        # Score using the pre-defined model
        scoring_response = {
            'predictions': [{'fields': ['nlp_prediction'], 
                             'values': scoring_prediction_out
                            }]
        }

        return scoring_response
    
    return score

## Save the Python function to the Model repository

In [ ]:
pyfunc_swspec_id = client.software_specifications.get_uid_by_name("runtime-23.1-py3.10")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'sentiment_function',
    client.repository.FunctionMetaNames.DESCRIPTION: 'sentiment_function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}


sentiment_function_details = client.repository.store_function(meta_props=meta_data, function=detect_sentiment)

## Create an Online Deployment

In [ ]:
sentiment_function_uid = client.repository.get_function_uid(sentiment_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
   client.deployments.ConfigurationMetaNames.ONLINE: {   }
}

sentiment_deployment_details = client.deployments.create(sentiment_function_uid, meta_props=meta_props)

sentiment_deployment_id = client.deployments.get_uid(sentiment_deployment_details)

## Predict sentiment of sample data

In [10]:
scoring_payload = {
   client.deployments.ScoringMetaNames.INPUT_DATA: [{  
        'fields': ["Text"],
        'values': [["The room is nice, but the price is too expensive. The window frame is made of wood."],
                   ["The check-in was horrible."],
                  ]
   }]
}

In [11]:
predictions = client.deployments.score(sentiment_deployment_id, scoring_payload)
predictions

{'predictions': [{'fields': ['nlp_prediction'],
   'values': [[{'targeted_sentiments': {'room': {'score': 0.9972254037857056,
        'label': 'SENT_POSITIVE',
        'mixed': False,
        'sentiment_mentions': [{'span': {'begin': 4, 'end': 8, 'text': 'room'},
          'sentimentprob': {'positive': 0.9972254037857056,
           'neutral': 0.0,
           'negative': 0.0027745962142944336}}]},
       'price': {'score': -0.9954081773757935,
        'label': 'SENT_NEGATIVE',
        'mixed': False,
        'sentiment_mentions': [{'span': {'begin': 26,
           'end': 31,
           'text': 'price'},
          'sentimentprob': {'positive': 0.004591822624206543,
           'neutral': 0.0,
           'negative': 0.9954081773757935}}]}},
      'producer_id': {'name': 'BERT TSA', 'version': '0.0.1'}}],
    [{'targeted_sentiments': {'check-in': {'score': -0.9992873072624207,
        'label': 'SENT_NEGATIVE',
        'mixed': False,
        'sentiment_mentions': [{'span': {'begin': 4,
   

## Clean up: Delete the deployment and the function

In [12]:
client.deployments.delete(sentiment_deployment_id)

'SUCCESS'

In [13]:
client.repository.delete(sentiment_function_uid)

'SUCCESS'